In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils


In [5]:
cap = cv2.VideoCapture(1,cv2.CAP_DSHOW)
#cap = cv2.VideoCapture("video_left_sliding_final.mp4")
#ret, frame = cap.read()


def track_yellow(frame,colorLower,colorUpper):
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, colorLower, colorUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    radius = 0
    center = (0,0)

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    if radius > 5:
        cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
        cv2.circle(frame, center, 2, (0, 0, 255), -1)
    
    return frame



def nothing(x):
    pass

cv2.namedWindow("Trackbars")
cv2.createTrackbar("Lower - H","Trackbars",0,179,nothing)
cv2.createTrackbar("Lower - S","Trackbars",0,255,nothing)
cv2.createTrackbar("Lower - V","Trackbars",0,255,nothing)
cv2.createTrackbar("Upper - H","Trackbars",179,179,nothing)
cv2.createTrackbar("Upper - S","Trackbars",255,255,nothing)
cv2.createTrackbar("Upper - V","Trackbars",255,255,nothing)

while True:
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    l_h = cv2.getTrackbarPos("Lower - H","Trackbars")
    l_s = cv2.getTrackbarPos("Lower - S","Trackbars")
    l_v = cv2.getTrackbarPos("Lower - V","Trackbars")
    u_h = cv2.getTrackbarPos("Upper - H","Trackbars")
    u_s = cv2.getTrackbarPos("Upper - S","Trackbars")
    u_v = cv2.getTrackbarPos("Upper - V","Trackbars")
    
    lowerColor = np.array([l_h, l_s, l_v])
    upperColor = np.array([u_h, u_s, u_v])
    
    mask = cv2.inRange(hsv, lowerColor, upperColor)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    result = cv2.bitwise_and(frame,frame,mask=mask)
    #result = track_yellow(frame,lowerColor,upperColor)
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    cv2.imshow("result",result)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break 
cap.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()